In [ ]:
!pip install -q ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.9/618.9 kB 6.1 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torchvision import models
from torch.nn.utils import prune
from torch.utils.data import DataLoader

In [ ]:
import time
from tqdm.notebook import tqdm

In [ ]:
model = YOLO('yolov8l.pt')

100%|██████████| 83.7M/83.7M [00:01<00:00, 82.4MB/s]


In [ ]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 166.848MB


In [ ]:
model.eval()

In [ ]:
#!yolo task=detect mode=val model=yolov8l.pt name=yolov8l_eval

WARNING ⚠️ 'data' is missing. Using default 'data=coco8.yaml'.
Ultralytics YOLOv8.0.190 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8l summary (fused): 268 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs

Dataset 'coco8.yaml' images not found ⚠️, missing path '/content/datasets/coco8/images/val'
100% 433k/433k [00:00<00:00, 14.0MB/s]
Unzipping /content/datasets/coco8.zip to /content/datasets/coco8...: 100% 25/25 [00:00<00:00, 2758.32file/s]
Dataset download success ✅ (0.7s), saved to /content/datasets

val: Scanning /content/datasets/coco8/labels/val... 4 images, 0 backgrounds, 0 corrupt: 100% 4/4 [00:00<00:00, 233.66it/s]
val: New cache created: /content/datasets/coco8/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:15<00:00, 15.39s/it]
                   all          4         17      0.914      0.839      0.976       0.76
                person          4         10          1       0

In [ ]:
inp = torch.randn(1, 3, 224, 224)

num_samples = 100
start_time = time.time()
for _ in tqdm(range(num_samples)):
    output = model(inp / 255)
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')

  0%|          | 0/100 [00:00<?, ?it/s]


0: 224x224 (no detections), 704.2ms
Speed: 0.0ms preprocess, 704.2ms inference, 13.4ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 511.4ms
Speed: 0.0ms preprocess, 511.4ms inference, 1.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 621.6ms
Speed: 0.0ms preprocess, 621.6ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 454.2ms
Speed: 0.0ms preprocess, 454.2ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 446.7ms
Speed: 0.0ms preprocess, 446.7ms inference, 2.0ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 411.2ms
Speed: 0.0ms preprocess, 411.2ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 444.7ms
Speed: 0.0ms preprocess, 444.7ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 417.1ms
Speed: 0.1ms prep

Avg inference time: 378.9708 ms


In [ ]:
infer_time = ((end_time - start_time) / num_samples) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')

Avg inference time: 378.9708 ms


In [ ]:
print(model)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): ModuleList(
          (0-2): 3 x Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
              (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (act): SiLU(inplac

# Квантизация

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Conv2d},
    dtype=torch.qint8
)

Ultralytics YOLOv8.0.194 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=coco8.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, 

In [ ]:
torch.save(quantized_model.state_dict(), 'quantized.pt')

In [ ]:
param_size = 0
for param in quantized_model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in quantized_model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('quantized_model size: {:.3f}MB'.format(size_all_mb))

quantized_model size: 166.848MB


In [ ]:
inp = torch.randn(1, 3, 224, 224)

num_samples = 100
start_time = time.time()
for _ in tqdm(range(num_samples)):
    output = quantized_model(inp)
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')

  0%|          | 0/100 [00:00<?, ?it/s]


WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.481132507324219. Dividing input by 255.
0: 224x224 (no detections), 417.3ms
Speed: 0.0ms preprocess, 417.3ms inference, 1.2ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.481132507324219. Dividing input by 255.
0: 224x224 (no detections), 307.1ms
Speed: 0.0ms preprocess, 307.1ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.481132507324219. Dividing input by 255.
0: 224x224 (no detections), 299.1ms
Speed: 0.0ms preprocess, 299.1ms inference, 1.1ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.481132507324219. Dividing input by 255.
0: 224x224 (no detections), 280.5ms
Speed: 0.0ms preprocess, 280.5ms inference, 0.7ms postprocess per image at shape (

Avg inference time: 322.8817 ms


# Прунинг

In [ ]:
from torch.nn.utils import prune

In [ ]:
for name, m in model.named_modules():
  params = list(m.named_parameters())
  if len(params) and params[0][0] == 'weight':
    prune.l1_unstructured(m, name=params[0][0], amount=0.3)

In [ ]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 333.072MB


In [ ]:
inp = torch.randn(1, 3, 224, 224)

num_samples = 100
start_time = time.time()
for _ in tqdm(range(num_samples)):
    output = model(inp / 255)
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')

  0%|          | 0/100 [00:00<?, ?it/s]


0: 224x224 (no detections), 354.1ms
Speed: 0.0ms preprocess, 354.1ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 305.3ms
Speed: 0.0ms preprocess, 305.3ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 307.9ms
Speed: 0.0ms preprocess, 307.9ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 312.6ms
Speed: 0.0ms preprocess, 312.6ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 307.9ms
Speed: 0.0ms preprocess, 307.9ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 314.2ms
Speed: 0.0ms preprocess, 314.2ms inference, 0.7ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 318.7ms
Speed: 0.0ms preprocess, 318.7ms inference, 0.8ms postprocess per image at shape (1, 3, 224, 224)

0: 224x224 (no detections), 311.4ms
Speed: 0.0ms prepr

Avg inference time: 346.0605 ms


In [ ]:
torch.save(model.state_dict(), 'pruned.pt')

In [ ]:
model.eval()

Ultralytics YOLOv8.0.194 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=coco8.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: []
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79bd2165e440>
fitness: 0.0
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([], dtype=float64)
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'k

# Кластеризация

In [ ]:
model = YOLO('yolov8l.pt')
tensors = model.state_dict()

In [ ]:
from sklearn.cluster import kmeans_plusplus
import numpy as np

tensors = model.state_dict()
for name in tensors:
  if name.find('weight') < 0:
    continue
  n = np.prod(tensors[name].shape)
  centroids = kmeans_plusplus(np.array(tensors[name].reshape(n, 1), dtype=float), 10)[0].squeeze()
  arr = tensors[name].reshape(n,)
  dist = abs(arr - centroids[0])
  for i in range(1, len(centroids)):
    dist = np.vstack((dist, abs(arr - centroids[i])))
  args_of_cent = dist.argmin(axis = 0)
  arr = centroids[args_of_cent]
  tensors[name] = torch.Tensor(arr.reshape(tensors[name].shape))
torch.save(tensors, 'clustered.pt')

In [ ]:
model = YOLO('yolov8l.pt')
model.load_state_dict(torch.load('clustered.pt'))

<All keys matched successfully>

In [ ]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 166.848MB


In [ ]:
inp = torch.randn(1, 3, 224, 224)

num_samples = 100
start_time = time.time()
for _ in tqdm(range(num_samples)):
    output = model(inp)
end_time = time.time()

infer_time = ((end_time - start_time) / num_samples) * 1000
print(f'Avg inference time: {infer_time:.4f} ms')

  0%|          | 0/100 [00:00<?, ?it/s]


WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.481132507324219. Dividing input by 255.
0: 224x224 (no detections), 309.3ms
Speed: 0.0ms preprocess, 309.3ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.481132507324219. Dividing input by 255.
0: 224x224 (no detections), 311.6ms
Speed: 0.0ms preprocess, 311.6ms inference, 1.0ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.481132507324219. Dividing input by 255.
0: 224x224 (no detections), 298.8ms
Speed: 0.0ms preprocess, 298.8ms inference, 0.9ms postprocess per image at shape (1, 3, 224, 224)

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.481132507324219. Dividing input by 255.
0: 224x224 (no detections), 299.7ms
Speed: 0.0ms preprocess, 299.7ms inference, 0.9ms postprocess per image at shape (

Avg inference time: 353.7584 ms


In [ ]:
model.eval()

Ultralytics YOLOv8.0.194 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=coco8.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0, 16, 17, 20, 25, 58])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79bd2274c160>
fitness: 0.0016112280701754385
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([  0.0069053,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,           0,           0,   0.0011509,   0.0011509,           0,   0.0011509,   0.0011509,   0.0011509,
         0.0011509,           0,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,   0.0011509,  

# Дистилляция

In [ ]:
!pip install -q \
autodistill \
autodistill-yolov8 \
roboflow \
supervision==0.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.0/527.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
import supervision as sv
from tqdm.notebook import tqdm
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO

In [ ]:
%mkdir /content/images
%mkdir /content/images/train
%mkdir /content/images/val

%mkdir /content/labels
%mkdir /content/labels/train
%mkdir /content/labels/val

In [ ]:
from roboflow import Roboflow

In [ ]:
rf = Roboflow(api_key="KPdQzbniHwGMSQyhZx76")
project = rf.workspace("surfacedefectdetectiondataset").project("animal-mflmf")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.195, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to animal-1 in yolov8:: 100%|██████████| 1536/1536 [00:00<00:00, 4439.59it/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/cat_dog_dataset/ /content/animal-1/train/

In [ ]:
target_model = YOLO("yolov8l.pt")
target_model.train(data = '/content/animal-1/data.yaml', epochs=15, batch = 32, pretrained = True)

Ultralytics YOLOv8.0.195 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/animal-1/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, opt

ModuleNotFoundError: ignored